In [1]:
        import numpy as np
        import gym

        from keras.models import Sequential, Model
        from keras.layers import Dense, Activation, Flatten, Input, Concatenate
        from keras.optimizers import Adam

        from rl.agents import DDPGAgent

        from rl.memory import SequentialMemory
        from rl.random import OrnsteinUhlenbeckProcess
        from rl.callbacks import FileLogger, ModelIntervalCheckpoint
        from keras.callbacks import TensorBoard




Using TensorFlow backend.


In [2]:
from subprocess import Popen, PIPE
import os
from tempfile import mkdtemp
from werkzeug import secure_filename
import requests
import gym
from gym import error, spaces, utils
from gym.utils import seeding
from gym import spaces, logger
import subprocess
from subprocess import Popen, PIPE
import numpy as np 
import time 
import csv
import pandas as pd 

class MicroEnv(gym.Env):
    metadata = {'render.modes': ['human']}
    def __init__(self):
        print('__init__')
        tstart = time.time()
        #filename1="dqn_logsv4.csv"
        #self.fileHandle = open(filename1,"w")
        #self.writer = csv.writer(self.fileHandle)
        self.maxNode = 2 
        self.minNode = 1 
        self.node = 1
        self.ip = '192.168.99.100'
        self.cluster= False
        #self.ip = subprocess.check_output(["docker-machine", "ip", "new-mgr"])
        self.cpu_axis  = self.get_cpu_observation()
        self.mem_axis = self.get_mem_observation()
        self.disk_axis = self.get_disk_observation()
        self.net_axis  = self.get_net_observation()
        self.action_space = spaces.Discrete(10)
        self.action_space = spaces.Box(0, 9, shape=(1, 1), dtype=np.int16)
        high = np.array([
            self.get_cpu_observation(),
            self.get_mem_observation(),
            self.get_disk_observation(),
            self.get_net_observation()])
        low = np.array([
            np.zeros(5),
            np.zeros(5),
            np.zeros(5),
            np.zeros(5)])
        self.observation_space = spaces.Box(low, high, dtype=np.float32)
        self.seed()
        self.obs= 0
        self.obs = self.get_observation()
        self.viewer = None
        self.state = self.get_observation()
        self.state_name = 's0'
        self.attempt = 0 
        self.steps_beyond_done = None
        self.done = False
        self.adapte_cpu= False  
        self.adapte_mem= False 
        self.adapte_disk= False 
        self.adapte_net= False 
        #self.writer.writerow(["timestamp","state", "action", "reward", "maxUtility", "duration", "info", "# node"])
    def seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]
    
    def step(self, action):
        #assert self.action_space.contains(action), "%r (%s) invalid"%(action, type(action))
        action = np.floor(action)
        state = self.get_observation()
        #past_stat = self.state
        #print(state)
        #Find the Utility Prefernces 
        # Select action 
        # get the reward value
        tstart = time.time()
        maxUtility = np.amax(self.obs[:,4])
        utilityType = np.argmax(self.obs[:,4])
        if utilityType== 0:
            self.adapte_cpu=True 
        elif utilityType== 1:
            self.adapte_mem = True
        elif utilityType== 2:
            self.adapte_disk = True
        elif utilityType== 3:
            self.adapte_net = True
        self.attempt += 1
        #print('self.adapte_cpu: ', self.adapte_cpu, 'self.adapte_mem:', self.adapte_mem, 'self.adapte_disk:',self.adapte_disk,'self.adapte_net:', self.adapte_net )
        done=False
        reward=0
        info=''
        print ("Starting Action: ", action)
        if action==0:
            #print("Stay in State S0")
            self.obs = self.get_observation()
            reward=  np.amax(self.obs[:,3])
            self.state_name='s0'
        elif action == 1:
            #print("Stay in State S1")
            self.obs = self.get_observation()
            done = True
            self.state_name='s1'
            reward=  1
        elif action == 2:
            self.obs = self.get_observation()
            try:
                response = requests.get('http://'+self.ip+':8888/services/vscale/web/'+ str(self.attempt) + '/' + str(self.cpu_axis[0])+'/'+str(self.cpu_axis[3]))
                results = response.json()
                if results['result']=='Service converged':
                    done=True 
                    self.obs = self.get_observation()
                    reward= 1 - np.amax(self.obs[:,3]) 
                    self.state_name='s2'
                    info = "Scale Up Move to State S2"
                else:
                    done= False
                    print(results)
                    self.obs = self.get_observation()
                    reward= 1- np.amax(self.obs[:,3])  
                    print("reward: ",reward, np.amax(self.obs[:,4]))
                    self.state_name='s0'
                    info = "S2 => No Scale Move back to State S0"
            except:
                pass
            finally:
                
                pass

        elif action == 3: 
            #print("Maintain Cluster State S4 and delete dangling docker containers")
            self.obs = self.get_observation()
            reward= 1 - np.amax(self.obs[:,3])
            #print("reward: ",reward, np.amax(self.obs[:,4]))
            done= False
            self.state_name='s0'
            info = "delete dangling docker containers S3"
            cur_dir = os.getcwd()
            filepath = os.path.join(cur_dir, 'cleancontainers.sh')
            print (filepath)
            res= subprocess.call(filepath, shell=True)
            #print (res)
            reward = 1
            self.state_name='s3'
            done = True
        elif action == 4:
            self.obs = self.get_observation()
            if (self.node <= self.maxNode):
                #print("Add Node s4")
                cur_dir = os.getcwd()
                filepath = os.path.join(cur_dir, 'addNode.sh')
                print (filepath)
                res= subprocess.call(filepath, shell=True)
                #print (res)
                info = "Add Node S4"
                self.obs = self.get_observation()
                reward= 1 - np.amax(self.obs[:,3])
                self.state_name='s4'
                print("reward: ",reward, np.amax(self.obs[:,4]))
                done= True
                self.node +=1
            elif self.node==5:
                    done=True 
                    reward= 1 
                    self.state_name='s4'
            else:
                #print("go back to Cluster State at S0: ", self.attempt)
                done= False
                self.obs = self.get_observation()
                reward= 1 - np.amax(self.obs[:,3])
                print("reward: ",reward, np.amax(self.obs[:,4]))
                self.state_name='s0'
        elif action == 5:
            self.obs = self.get_observation()
            if (self.node <= self.maxNode and self.node > self.minNode ):
                #print("Delete Node S5")
                self.state_name='s5'
                cur_dir = os.getcwd()
                filepath = os.path.join(cur_dir, 'deleteNode.sh')
                #print (filepath)
                res= subprocess.call(filepath, shell=True)
                #print (res)
                
                info = "Delete Node S5"
                reward= 1
                #print("reward: ",reward, np.amax(self.obs[:,4]))
                done= True
                self.node -=1
            else:
                #print("Maintain Cluster State at S0: ", self.attempt, self.node, self.minNode, self.maxNode)
                done= False
                reward= 1 - np.amax(self.obs[:,3])
                #print("reward: ",reward)
                self.state_name='s0'
                if self.node==1:
                    done=True 
                    reward= 1
                    self.state_name='s5'
        elif action==6:
            self.obs = self.get_observation()
            #print("freedisk Space S6")
            cur_dir = os.getcwd()
            filepath = os.path.join(cur_dir, 'freedisk.sh')
            #print (filepath)
            res= subprocess.call(filepath, shell=True)
            #print (res)
            info = "freedisk Node S6"
            self.state_name='s6'
            reward= 1
            #print("reward: ",reward, np.amax(self.obs[:,4]))
            done= True
        
        elif action == 7:
            self.obs = self.get_observation()
            if (self.node <= self.maxNode and self.node >= self.minNode ):
                self.state_name='s7'
                cur_dir = os.getcwd()
                filepath = os.path.join(cur_dir, 'promoteNode.sh')
                #print (filepath)
                res= subprocess.call(filepath, shell=True)
                #print (res)
                info = "Promote Worker node to Manager S7"
                reward= 1
                #print("reward: ",reward, np.amax(self.obs[:,4]), "Node: ", self.node)
                done= True
            elif (self.node >= self.maxNode):
                info = "Maintain Manager nodes S0"
                self.state_name='s0'
                reward= 1 - np.amax(self.obs[:,3])
                #print("reward: ",reward, np.amax(self.obs[:,4]), "Node: ", self.node)
                done= True 
        elif action==8:
            self.obs = self.get_observation()
            if (self.node <= self.maxNode):
                self.state_name='s8'
                cur_dir = os.getcwd()
                filepath = os.path.join(cur_dir, 'manager.sh')
                #print (filepath)
                res= subprocess.call(filepath, shell=True)
                #print (res)
                info = "Add Manager node S8"
                self.node += 1
                reward= 1
                #print("reward: ",reward, np.amax(self.obs[:,4]), "Node: ", self.node)
                done= True
            elif (self.node >= self.maxNode):
                info = "Maintain Manager nodes S0"
                self.state_name='s8'
                reward= 1 - np.amax(self.obs[:,3])
                #print("reward: ",reward, np.amax(self.obs[:,4]), "Node: ", self.node)
                done= True 
            else:
                #print("Maintain Cluster State S0", self.attempt, self.node )
                self.state_name='s0'
                reward= 1 - np.amax(self.obs[:,3])
                done= False
                info = "from S8 to S0 "
                
        elif action == 9:
            self.obs = self.get_observation()
            if not self.cluster:
                #add new cluster one time only
                try:         
                    print("roll-back and enforce new cluster")
                    cur_dir = os.getcwd()
                    filepath = os.path.join(cur_dir, 'nupicnewcluster.sh')
                    print (filepath)
                    res= subprocess.call(filepath, shell=True)
                     
                    self.state_name='s9'
                    info = "rollback and enforce new  cluster S9"
                    #time.sleep(300)
                    myip = subprocess.check_output(["docker-machine", "ip", "nupic"])
                    self.ip=myip.decode('utf-8').strip()
                    reward= 1
                    #print("reward: ",reward, np.amax(self.obs[:,4]))
                    done= True
                    self.cluster = True
                    #print(reward)
                    info = "roll-back and enforce new cluster"
                except:
                    pass
                finally:
                    pass
            elif self.cluster:
                self.state_name='s9'
                reward= 1 - np.amax(self.obs[:,3])
                done= False
                info = "from S9 to S0 "
                if self.attempt > 400: 
                    done= True 
                    info = "Exit S9 to S0 "
                    reward= 1 - np.amax(self.obs[:,3])

            
        else: 
            print ("action not defined")
            self.state_name='s0'
            self.obs = self.get_observation()
            done= False
            reward= -1  
            info = "action not defined"
        if done: 
            reward = 1
        elif self.steps_beyond_done is None:
            #Adaptation Failed 
            reward = 0.0 
            self.steps_beyond_done = 0
        else: 
            if self.steps_beyond_done == 1:
                logger.warn("You are calling 'step()' even though this environment has already returned done = True. You should always call 'reset()' once you receive 'done = True' -- any further steps are undefined behavior.")
                self.steps_beyond_done += 1
                reward = 0.0
        tend = tstart - time.time()
        print ("\nState: ", self.state_name, "action: ", action, "reward: ", reward, "# nodes:", self.node, "cluster: ", self.cluster )
        #writer1.writerow(["timestamp","state", "action", "reward", "maxUtility", "duration"])
        self.obs = self.get_observation()
        #self.writer.writerow([tstart,self.state_name, action, reward, np.amax(self.obs[:,4]), tend, info, self.node])
        return self.obs, reward, done, {}
    def reset(self):
        #self.fileHandle.close()
        self.state = self.get_observation()
        self.steps_beyond_done = None
        self.adapte_cpu= False  
        self.adapte_mem= False 
        self.adapte_disk= False 
        self.adapte_net= False
        cur_dir = os.getcwd()
        filepath = os.path.join(cur_dir, 'deleteallnodes.sh')
        print (filepath)
        self.maxNode = 3 
        self.minNode = 1 
        #self.node = 1
        #self.cluster= False
        return self.state
    def render(self, mode='human', close=False):
        logger.warn("View is not allowed in this environment")
        return 0 
    def close(self):
        self.fileHandle.close()
        if self.viewer:
            self.viewer.close()
            self.viewer = None
    def get_observation(self):
        try:
            self.disk_axis = self.get_disk_observation() 
            self.mem_axis = self.get_mem_observation()
            self.cpu_axis = self.get_cpu_observation()
            self.net_axis = self.get_net_observation()
            obs = np.vstack((self.cpu_axis,self.mem_axis, self.disk_axis, self.net_axis) )
            return obs
        except:
            pass
        finally: 
            pass 
    def get_cpu_observation(self):
        try: 
            response = requests.get('http://'+self.ip+':8888/cpu', timeout=5)
            results = response.json()
            if len(results) > 0:
                    cpu = results['cpu']
                    prediction = results['prediction']
                    anomalyScore = results['anomalyScore']
                    anomalyLikelihood = results['anomalyLikelihood']
                    utility_cpu = results['utility_cpu']
                    cpu_axis=[cpu, prediction, anomalyScore, anomalyLikelihood, utility_cpu]
        except:
            cpu_axis=[-1, -1, -1, -1, -1]
            pass    
        finally: 
            pass
        return np.array(cpu_axis)
    def get_mem_observation(self):
        try:
            response = requests.get('http://'+self.ip+':8888/mem', timeout=5)
            results = response.json()
            if len(results) > 0:
                mem = results['mem']
                prediction = results['prediction']
                anomalyScore = results['anomalyScore']
                anomalyLikelihood = results['anomalyLikelihood']
                utility_mem = results['utility_mem']
                mem_axis=[mem, prediction, anomalyScore, anomalyLikelihood, utility_mem]
        except:
            mem_axis=[-1, -1, -1, -1, -1]
            pass     
        finally: 
            #mem_axis=[0, 0, 0, 0, 0]
            pass
        return np.array(mem_axis)
    def get_net_observation(self):
        try:
            response = requests.get('http://'+self.ip+':8888/net', timeout=5)
            results = response.json()
            if len(results) > 0:
                net = results['net']
                prediction = results['prediction']
                anomalyScore = results['anomalyScore']
                anomalyLikelihood = results['anomalyLikelihood']
                utility_net = results['utility_net']
                net_axis=[net, prediction, anomalyScore, anomalyLikelihood, utility_net]
        except:
            net_axis=[-1, -1, -1, -1, -1]
            pass
        finally:
            pass 
        return np.array(net_axis)
    def get_disk_observation(self):
        try:
            response = requests.get('http://'+self.ip+':8888/disk', timeout=5)
            if response is not None:
                results = response.json()
                if len(results) > 0:
                    disk = results['disk']
                    prediction = results['prediction']
                    anomalyScore = results['anomalyScore']
                    anomalyLikelihood = results['anomalyLikelihood']
                    utility_disk = results['utility_disk']
            disk_axis=[disk, prediction, anomalyScore, anomalyLikelihood, utility_disk]
        except:
            disk_axis=[-1, -1, -1, -1, -1]
            pass
        finally:
            
            pass
        return np.array(disk_axis)

    def get_current_state(self):
        current_state = self.state
         
        return current_state
     

In [3]:
from gym import spaces
import numpy as np
space = spaces.Discrete(10) 
space = spaces.Box(0, 10, shape=(1, 1), dtype=np.int16)

In [4]:
#import gym_microagent
import gym_microagent
ENV_NAME = 'microagent-v20'
 

In [5]:
# Get the environment and extract the number of actions.
#env = gym.make(ENV_NAME)
env = MicroEnv()
np.random.seed(123)
env.seed(123)


__init__


[123]

In [6]:
env.action_space.shape

(1, 1)

In [7]:
env.action_space.shape[0]

1

In [8]:
assert len(env.action_space.shape) == 2
nb_actions = env.action_space.shape[0]

In [9]:
nb_actions

1

In [10]:

# Next, we build a very simple model.
actor = Sequential()
actor.add(Flatten(input_shape=(1,) + env.observation_space.shape))
actor.add(Dense(16, activation='relu'))

actor.add(Dense(16, activation='relu'))

actor.add(Dense(16, activation='relu'))

actor.add(Dense(nb_actions, activation='linear'))

print(actor.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 20)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                336       
_________________________________________________________________
dense_2 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_3 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 17        
Total params: 897
Trainable params: 897
Non-trainable params: 0
_________________________________________________________________
None


In [12]:
action_input = Input(shape=(nb_actions,), name='action_input')
observation_input = Input(shape=(1,) + env.observation_space.shape, name='observation_input')
flattened_observation = Flatten()(observation_input)

In [13]:
x = Concatenate()([action_input, flattened_observation])
x = Dense(32, activation='relu')(x)

x = Dense(32, activation='relu')(x)

x = Dense(32, activation='relu')(x)

x = Dense(1, activation='linear')(x)

critic = Model(inputs=[action_input, observation_input], outputs=x)
print(critic.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
observation_input (InputLayer)  (None, 1, 4, 5)      0                                            
__________________________________________________________________________________________________
action_input (InputLayer)       (None, 1)            0                                            
__________________________________________________________________________________________________
flatten_2 (Flatten)             (None, 20)           0           observation_input[0][0]          
__________________________________________________________________________________________________
concatenate_2 (Concatenate)     (None, 21)           0           action_input[0][0]               
                                                                 flatten_2[0][0]                  
__________

In [14]:
from keras.utils import plot_model
plot_model(critic, show_shapes=True, expand_nested=True, rankdir='TB', to_file='critic.png')
plot_model(actor, show_shapes=True, expand_nested=True, rankdir='TB', to_file='actor.png')

In [49]:
memory = SequentialMemory(limit=5000, window_length=1)
random_process = OrnsteinUhlenbeckProcess(size=nb_actions, theta=.15, mu=0., sigma=.3)
agent = DDPGAgent(nb_actions=nb_actions, actor=actor, critic=critic, critic_action_input=action_input,
                  memory=memory, nb_steps_warmup_critic=100, nb_steps_warmup_actor=100,
                  random_process=random_process, gamma=.99, target_model_update=1e-3)
agent.compile(Adam(lr=.001, clipnorm=1.), metrics=['mae'])

In [50]:
%%time
weights_filename = 'ddpg{}_weights.h5f'.format(ENV_NAME)
checkpoint_weights_filename = 'ddpg_' + ENV_NAME + '_weights_{step}.h5f'
log_filename = 'ddpg_{}_log.json'.format(ENV_NAME)
callbacks = [ModelIntervalCheckpoint(checkpoint_weights_filename, interval=25000)]
callbacks += [FileLogger(log_filename, interval=25000)]
callbacks += [TensorBoard(log_dir='./ddpg_logsv2/', histogram_freq=0, write_graph=False)]
agent.fit(env, nb_steps=5000, callbacks=callbacks, visualize=False, verbose=1, nb_max_episode_steps=500)

Training for 5000 steps ...
/Users/baz/ieee-demo/deleteallnodes.sh
Interval 1 (0 steps performed)
Starting Action:  [1.]

State:  s1 action:  [1.] reward:  1 # nodes: 1 cluster:  False
    1/10000 [..............................] - ETA: 7:59:06 - reward: 1.0000/Users/baz/ieee-demo/deleteallnodes.sh
Starting Action:  [0.]

State:  s0 action:  [0.] reward:  0.0 # nodes: 1 cluster:  False
    2/10000 [..............................] - ETA: 8:59:15 - reward: 0.5000Starting Action:  [0.]

State:  s0 action:  [0.] reward:  -1 # nodes: 1 cluster:  False
    3/10000 [..............................] - ETA: 8:29:16 - reward: 0.0000e+00Starting Action:  [0.]

State:  s0 action:  [0.] reward:  -1 # nodes: 1 cluster:  False
    4/10000 [..............................] - ETA: 8:13:54 - reward: -0.2500   Starting Action:  [0.]

State:  s0 action:  [0.] reward:  -1 # nodes: 1 cluster:  False
    5/10000 [..............................] - ETA: 8:04:45 - reward: -0.4000Starting Action:  [0.]

State:  s0


State:  s0 action:  [402.] reward:  -1 # nodes: 1 cluster:  False
   98/10000 [..............................] - ETA: 7:49:23 - reward: -0.8622Starting Action:  [-1.]
action not defined

State:  s0 action:  [-1.] reward:  -1 # nodes: 1 cluster:  False
   99/10000 [..............................] - ETA: 7:47:06 - reward: -0.8636Starting Action:  [-8.]
action not defined

State:  s0 action:  [-8.] reward:  -1 # nodes: 1 cluster:  False
  100/10000 [..............................] - ETA: 7:46:08 - reward: -0.8650Starting Action:  [109.]
action not defined

State:  s0 action:  [109.] reward:  -1 # nodes: 1 cluster:  False
  101/10000 [..............................] - ETA: 7:44:15 - reward: -0.8663Starting Action:  [-8.]
action not defined

State:  s0 action:  [-8.] reward:  -1 # nodes: 1 cluster:  False
  102/10000 [..............................] - ETA: 7:48:34 - reward: -0.8676Starting Action:  [4.]
/Users/baz/ieee-demo/addNode.sh
reward:  0.5 49.0

State:  s4 action:  [4.] reward:  1 

  141/10000 [..............................] - ETA: 9:53:21 - reward: -0.8298Starting Action:  [33.]
action not defined

State:  s0 action:  [33.] reward:  -1 # nodes: 3 cluster:  False
  142/10000 [..............................] - ETA: 9:50:54 - reward: -0.8310Starting Action:  [-4.]
action not defined

State:  s0 action:  [-4.] reward:  -1 # nodes: 3 cluster:  False
  143/10000 [..............................] - ETA: 9:49:39 - reward: -0.8322Starting Action:  [-4.]
action not defined

State:  s0 action:  [-4.] reward:  -1 # nodes: 3 cluster:  False
  144/10000 [..............................] - ETA: 9:47:47 - reward: -0.8333Starting Action:  [10.]
action not defined

State:  s0 action:  [10.] reward:  -1 # nodes: 3 cluster:  False
  145/10000 [..............................] - ETA: 9:47:05 - reward: -0.8345Starting Action:  [7.]

State:  s7 action:  [7.] reward:  1 # nodes: 3 cluster:  False
  146/10000 [..............................] - ETA: 9:46:33 - reward: -0.8219/Users/baz/ieee

  226/10000 [..............................] - ETA: 10:13:53 - reward: -0.2345/Users/baz/ieee-demo/deleteallnodes.sh
Starting Action:  [3.]
/Users/baz/ieee-demo/cleancontainers.sh

State:  s3 action:  [3.] reward:  1 # nodes: 4 cluster:  True
  227/10000 [..............................] - ETA: 10:12:33 - reward: -0.2291/Users/baz/ieee-demo/deleteallnodes.sh
Starting Action:  [3.]
/Users/baz/ieee-demo/cleancontainers.sh

State:  s3 action:  [3.] reward:  1 # nodes: 4 cluster:  True
  228/10000 [..............................] - ETA: 10:11:11 - reward: -0.2237/Users/baz/ieee-demo/deleteallnodes.sh
Starting Action:  [3.]
/Users/baz/ieee-demo/cleancontainers.sh

State:  s3 action:  [3.] reward:  1 # nodes: 4 cluster:  True
  229/10000 [..............................] - ETA: 10:09:51 - reward: -0.2183/Users/baz/ieee-demo/deleteallnodes.sh
Starting Action:  [3.]
/Users/baz/ieee-demo/cleancontainers.sh

State:  s3 action:  [3.] reward:  1 # nodes: 4 cluster:  True
  230/10000 [...............

  315/10000 [..............................] - ETA: 8:33:49 - reward: 0.2413Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
  316/10000 [..............................] - ETA: 8:33:03 - reward: 0.2468Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
  317/10000 [..............................] - ETA: 8:32:14 - reward: 0.2524Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
  318/10000 [..............................] - ETA: 8:31:28 - reward: 0.2579Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
  319/10000 [..............................] - ETA: 8:30:40 - reward: 0.2633Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
  320/10000 [..............................] - ETA: 8:29:52 - reward: 0.2687Starting Action:  [4.]
reward:  2 -1

State:

...


State:  s0 action:  [5.] reward:  2 # nodes: 4 cluster:  True
 4684/10000 [=============>................] - ETA: 2:30:00 - reward: 1.8247Starting Action:  [5.]

State:  s0 action:  [5.] reward:  2 # nodes: 4 cluster:  True
 4685/10000 [=============>................] - ETA: 2:29:58 - reward: 1.8248Starting Action:  [5.]

State:  s0 action:  [5.] reward:  2 # nodes: 4 cluster:  True
 4686/10000 [=============>................] - ETA: 2:29:56 - reward: 1.8248Starting Action:  [5.]

State:  s0 action:  [5.] reward:  2 # nodes: 4 cluster:  True
 4687/10000 [=============>................] - ETA: 2:29:54 - reward: 1.8248Starting Action:  [5.]

State:  s0 action:  [5.] reward:  2 # nodes: 4 cluster:  True
 4688/10000 [=============>................] - ETA: 2:29:52 - reward: 1.8249Starting Action:  [5.]

State:  s0 action:  [5.] reward:  2 # nodes: 4 cluster:  True
 4689/10000 [=============>................] - ETA: 2:29:50 - reward: 1.8249Starting Action:  [5.]

State:  s0 action:  [5.] re

 4784/10000 [=============>................] - ETA: 2:26:31 - reward: 1.8284Starting Action:  [5.]

State:  s0 action:  [5.] reward:  2 # nodes: 4 cluster:  True
 4785/10000 [=============>................] - ETA: 2:26:29 - reward: 1.8284Starting Action:  [5.]

State:  s0 action:  [5.] reward:  2 # nodes: 4 cluster:  True
 4786/10000 [=============>................] - ETA: 2:26:26 - reward: 1.8285Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
 4787/10000 [=============>................] - ETA: 2:26:25 - reward: 1.8285Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
 4788/10000 [=============>................] - ETA: 2:26:23 - reward: 1.8285Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
 4789/10000 [=============>................] - ETA: 2:26:21 - reward: 1.8286Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  

 4832/10000 [=============>................] - ETA: 2:25:04 - reward: 1.8301Starting Action:  [5.]

State:  s0 action:  [5.] reward:  2 # nodes: 4 cluster:  True
 4833/10000 [=============>................] - ETA: 2:25:02 - reward: 1.8301/Users/baz/ieee-demo/deleteallnodes.sh
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  0.0 # nodes: 4 cluster:  True
 4834/10000 [=============>................] - ETA: 2:25:01 - reward: 1.8297Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
 4835/10000 [=============>................] - ETA: 2:25:00 - reward: 1.8298Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
 4836/10000 [=============>................] - ETA: 2:24:58 - reward: 1.8298Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
 4837/10000 [=============>................] - ETA: 2:24:56 - reward: 1.8299Starting Action: 

 4923/10000 [=============>................] - ETA: 2:22:36 - reward: 1.8318Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
 4924/10000 [=============>................] - ETA: 2:22:35 - reward: 1.8318Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
 4925/10000 [=============>................] - ETA: 2:22:33 - reward: 1.8319Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
 4926/10000 [=============>................] - ETA: 2:22:31 - reward: 1.8319Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
 4927/10000 [=============>................] - ETA: 2:22:30 - reward: 1.8319Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
 4928/10000 [=============>................] - ETA: 2:22:28 - reward: 1.8320Starting Action:  [4.]
reward:  2 -1

State:

reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
 4970/10000 [=============>................] - ETA: 2:21:19 - reward: 1.8334Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
 4971/10000 [=============>................] - ETA: 2:21:17 - reward: 1.8334Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
 4972/10000 [=============>................] - ETA: 2:21:16 - reward: 1.8335Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
 4973/10000 [=============>................] - ETA: 2:21:14 - reward: 1.8335Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
 4974/10000 [=============>................] - ETA: 2:21:12 - reward: 1.8335Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
 4975/10000 [=============>................

In [51]:
agent.save_weights('ddpg_{}_weights.h5f'.format(ENV_NAME), overwrite=True)


In [52]:
agent.test(env, nb_episodes=5, visualize=False, nb_max_episode_steps=200)


Testing for 5 episodes ...
/Users/baz/ieee-demo/deleteallnodes.sh
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  0.0 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  

Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True


Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True


reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]


reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]


...

reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]


reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]


Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True


Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True


Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
Starting Action:  [4.]
reward:  2 -1

State:  s0 action:  [4.] reward:  2 # nodes: 4 cluster:  True
